<a href="https://colab.research.google.com/github/hyounghe0724/StartPytorch/blob/main/DataLoader_customization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch
import torchvision.transforms as tr # 전처리 기능
from torch.utils.data import DataLoader, Dataset # 데이터를 모델에 사용 할 수 있도록 정리해 주는 라이브러리
import numpy as np

In [9]:
# 32x32 컬러 이미지와 라벨이 각각 100장
# glob -> PIL, OpenCV ...
train_images = np.random.randint(256,size=(100, 32,32,3))
train_labels = np.random.randint(256,size=(100, 1))

# if using OpenCV, develop this here

In [12]:
class MyDataset(Dataset):
  def __init__(self, x_data, y_data, transform=None):

    self.x_data = x_data # numpy 배열
    self.y_data = y_data # numpy 배열
    self.transform = transform
    self.len = len(y_data)

  def __getitem__(self, index):
    sample = self.x_data[index], self.y_data[index]

    if self.transform:
      sample = self.transform(sample) # transform이 None아니라면 전처리 작업 수행
    return sample

  def __len__(self):
    return self.len



In [11]:
# 전처리 기술을 직접 만들어 보자
# 이 때 위 기본 양식과 같이 사용하기 위해 call함수를 사용한다
# def __call__ 내의 원하는 전처리 작업을 프로그래밍 할 수 있다.

# 1. 텐서 변환
class ToTensor:
  def __call__(self, sample):
    inputs, labels = sample
    inputs = torch.FloatTensor(inputs) # 텐서 변환
    inputs = inputs.permute(2, 0 ,1) # 크기 변환
    return inputs,torch.LongTensor(labels) # 텐서 변환
# 2. 선형식
class LinearTensor:

  def __init__(self, slope=1, bias = 0):
    self.slope = slope
    self.bias = bias

  def __call__(self, sample):
    inputs, labels = sample
    inputs = self.slope*inputs + self.bias #ax + b
    return inputs, labels

  # 추가로 원하는 전처기를 작성
  # ..




In [13]:
trans = tr.Compose([ToTensor(), LinearTensor(2,5)])  # 텐서 변환 후 선형식 2x+5 연산
dataset1 = MyDataset(train_images, train_labels, transform=trans)
train_loader1 = DataLoader(dataset1, batch_size=10, shuffle=True)

# ToTensor()와 tr.Tensor()의 차이
# tr.Tensor()는 라이브러리를 사용
# ToTensor는 우리가 직접 만든 클래스 사용

In [17]:
dataiter1= iter(train_loader1)
images1, labels1 = next(dataiter1)
print(images1.size())


torch.Size([10, 3, 32, 32])


In [18]:
# torchvision.transforms은 입력 이미지가 일반적으로 PILImage 타입이나 텐서일 경우에 동작한다.
# 현재 데이터는 넘파이 배열이므로, 텐서 변환후, tr.ToPILImage()을 이용하여 PILImage 타입으로 변환
# __call__을 사용

class MyTransforms:

  def __call__(self, sample):
    inputs, labels = sample
    inputs = torch.FloatTensor(inputs)
    inputs = inputs.permute(2,0, 1)
    labels = torch.LongTensor(labels)

    transf = tr.Compose([tr.ToPILImage(), tr.Resize(128), tr.ToTensor(), tr.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])
    # tr.Normalize(mean, std)
    # (value - mean) / std -> normalization
    # (0.5,0.5,0.5) RGB채널의 평균값
    # (0.5, 0.5,0.5) 각각의 표준편차
    # 주로 (0.5,0.5,0.5)로 정규화를 진행하고
    # 이미지를 다시 복구시켜 출력하고 싶으면 다음과 같이 수정한다.
    # plt.imshow(transforms.ToPILImage()(image*0.5+0.5))
    # norm * 0.5 + 0.5 = original
    final_output = transf(inputs)

    return final_output, labels

In [22]:
dataset2 = MyDataset(train_images, train_labels, transform=MyTransforms())
train_loader2 = DataLoader(dataset2, batch_size=15, shuffle=True)


In [24]:
dataiter2 = iter(train_loader2)
images2, labels2 = next(dataiter2)
print(images2)

tensor([[[[-1.0000, -1.0000, -0.9373,  ...,  0.0039, -0.0353, -0.0353],
          [-1.0000, -1.0000, -0.9373,  ...,  0.0039, -0.0353, -0.0353],
          [-0.7725, -0.7725, -0.7412,  ...,  0.0353, -0.0118, -0.0118],
          ...,
          [-0.2235, -0.2235, -0.2471,  ...,  0.4980,  0.6706,  0.6706],
          [-0.3333, -0.3333, -0.3647,  ...,  0.6078,  0.8039,  0.8039],
          [-0.3333, -0.3333, -0.3647,  ...,  0.6078,  0.8039,  0.8039]],

         [[ 0.7255,  0.7255,  0.6314,  ..., -0.2863, -0.2471, -0.2471],
          [ 0.7255,  0.7255,  0.6314,  ..., -0.2863, -0.2471, -0.2471],
          [ 0.7412,  0.7412,  0.6314,  ..., -0.3020, -0.2706, -0.2706],
          ...,
          [ 0.1686,  0.1686,  0.0745,  ..., -0.2627, -0.2549, -0.2549],
          [ 0.0588,  0.0588, -0.0196,  ..., -0.3804, -0.4039, -0.4039],
          [ 0.0588,  0.0588, -0.0196,  ..., -0.3804, -0.4039, -0.4039]],

         [[ 1.0000,  1.0000,  0.9922,  ..., -0.1373, -0.2314, -0.2314],
          [ 1.0000,  1.0000,  